Import packages

In [2]:
import sys
sys.path.append('../helpers/')

import pickle

from pathlib import Path
import numpy as np
import os
from glob import glob
import natsort
import cv2
import csv
import random
from tqdm import tqdm

import matplotlib.pyplot as plt

# importing packages
from preprocessing import Preprocessing
from feature_extraction import FeatureExtraction

preprocessor = Preprocessing()
feature_extractor = FeatureExtraction()

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

Read the images

In [3]:
# SAVE_PATH = Path("Processed-Heart-Detection")
# dcm_path = ROOT_PATH/str(patient_id) # joining paths
# dcm_path = dcm_path.with_suffix(".dcm") # adds suffix

ROOT_PATH = Path(Path(os.getcwd())/"../challenge1")
TRAIN_PATH = ROOT_PATH/"train"
VAL_PATH = ROOT_PATH/"val"

print(TRAIN_PATH)

c:\master\udg\CAD\project\notebooks\..\challenge1\train


In [4]:
train_nevus = sorted(glob(str(TRAIN_PATH/'nevus/*')))
train_others = sorted(glob(str(TRAIN_PATH/'others/*')))

val_nevus = sorted(glob(str(VAL_PATH/'nevus/*')))
val_others = sorted(glob(str(VAL_PATH/'others/*')))


In [4]:
# train_nevus

# TODO ASK:
# Do we normalize the images in pre-processing or after we extract all of the features?
# Should we remove the hair? Aggressivly? 
# Should we fill the borders?

In [5]:
def display_img(image, gray=False):
    if gray:
        plt.imshow(image, cmap='gray')
    else:
        # plt.imshow(image)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')


In [15]:
path = os.getcwd()+'\..\output\nevus_train_features.pkl'
print(path)

c:\master\udg\CAD\project\notebooks\..\output
evus_train_features.pkl


In [ ]:
# Pre-processing and feature extraction
features_dir = r'../output/features/'

experiment              = 2
subsample               = True

nevus_train_filename    = f'{experiment}_nevus_train_features.csv'
nevus_val_filename      = f'{experiment}_nevus_val_features.csv'

others_train_filename   = f'{experiment}_others_train_features.csv'
others_val_filename     = f'{experiment}_others_val_features.csv'

#preprocessed_images_filename = Path(os.getcwd())/'output/preprocessed_images.pkl'

random.seed(42)
random.shuffle(train_nevus)
random.shuffle(val_nevus)
random.shuffle(train_others)
random.shuffle(val_others)


filenames_list = [nevus_train_filename, nevus_val_filename, others_train_filename,  others_val_filename]
dir_list = [train_nevus, val_nevus, train_others, val_others]

labels = [0, 0, 1, 1]

preprocessed_images = []

for index, filename in enumerate(filenames_list):
    with open(os.path.join(features_dir, filename), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        for count, image_path in tqdm(enumerate(dir_list[index])):

            if subsample:
                if count == 999: # only 1k per class
                    break

            # reading the image 
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)

            # 1. Copping black frame
            image_without_black_frame, _ = preprocessor.crop_frame(image)

            # 2. Resizing
            image_resized = preprocessor.resize_images(image_without_black_frame, preserve_ratio=True)

            # 3. Removing hair
            image_without_hair = preprocessor.extract_hair(image_resized)

            # Saving the preprocessed image to a list
            preprocessed_images.append(image_without_hair)

            # 4. Displaying result
            # display_img(image_without_hair)
            
            # 5. Extracting features
            feature_vector = feature_extractor.fit(image_without_hair)

            # 6. Add label column
            feature_vector = np.append(feature_vector,labels[index])

            # print(feature_vector)
            writer.writerow(feature_vector)

### Saving pickles of preprocessing images

In [5]:
# Pre-processing and feature extraction
prep_imgs_dir = r'../output/'

nevus_train_prep_filename    = 'nevus_train_prep_images.pkl'
nevus_val_prep_filename      = 'nevus_val_prep_images.pkl'

others_train_prep_filename   = 'others_train_prep_images.pkl'
others_val_prep_filename     = 'others_val_prep_images.pkl'


filenames_prep_list = [nevus_train_prep_filename, nevus_val_prep_filename, others_train_prep_filename,  others_val_prep_filename]
dir_list = [train_nevus, val_nevus, train_others, val_others]




In [ ]:
subsample               = False

for index, filename in enumerate(filenames_prep_list):

    preprocessed_images = []

    for count, image_path in tqdm(enumerate(dir_list[index])):

        if subsample:
            if count == 999: # only 1k per class
                break

        # reading the image 
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)

        # 1. Copping black frame
        image_without_black_frame, _ = preprocessor.crop_frame(image)

        # 2. Resizing
        image_resized = preprocessor.resize_images(image_without_black_frame, preserve_ratio=True)

        # 3. Removing hair
        image_without_hair = preprocessor.extract_hair(image_resized)

        # Saving the preprocessed image to a list
        preprocessed_images.append(image_without_hair)

    # Saving the preprocessed images to a file
    with open(prep_imgs_dir+filename, 'wb') as file:
        pickle.dump(preprocessed_images, file)

### Feature extraction from saved images

In [6]:
# Pre-processing and feature extraction
features_dir = r'../output/features/'

experiment              = 3
subsample               = False

nevus_train_filename    = f'{experiment}_nevus_train_features.csv'
nevus_val_filename      = f'{experiment}_nevus_val_features.csv'

others_train_filename   = f'{experiment}_others_train_features.csv'
others_val_filename     = f'{experiment}_others_val_features.csv'

filenames_list = [nevus_train_filename, nevus_val_filename, others_train_filename, others_val_filename]
#images_lists = [nevus_train_prep_images, nevus_val_prep_images, others_train_prep_images, others_val_prep_images]

labels = [0, 0, 1, 1]


for pickle_filename, filename, label in zip(filenames_prep_list, filenames_list, labels):
    pickle_path = os.path.join(prep_imgs_dir, pickle_filename)

    with open(pickle_path, 'rb') as file:
        # Use a generator to load and yield images one at a time
        def image_generator():
            count = 0
            try:
                while True:
                    image_list = pickle.load(file)
                    for image in image_list:
                        yield image
                        count += 1
                        if subsample and count == 999:  # Only 1k per class
                            return
            except EOFError:
                pass

        with open(os.path.join(features_dir, filename), 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)

            count = 0
            for preprocessed_image in tqdm(image_generator(), desc=f"Processing {filename}"):
                if subsample and count == 9:  # Only 1k per class
                    break

                # 5. Extracting features
                feature_vector = feature_extractor.fit(preprocessed_image)

                # 6. Add label column
                feature_vector = np.append(feature_vector, label)

                # Write the feature vector to the CSV file
                writer.writerow(feature_vector)

                count += 1

Processing 3_nevus_train_features.csv: 7725it [1:46:51,  1.20it/s]
Processing 3_nevus_val_features.csv: 1931it [33:07,  1.03s/it]
Processing 3_others_train_features.csv: 7470it [2:47:59,  1.35s/it]
Processing 3_others_val_features.csv: 1865it [36:33,  1.18s/it]


In [ ]:
# Image augmentation
# 1. Create dataset class (DL)
# 2. pass the augmentation as an argument (create class/file)